In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!wget https://www.dropbox.com/s/e9nqvqgvuzaxdi4/TON_IoT.csv?dl=0

--2024-05-16 18:03:18--  https://www.dropbox.com/s/e9nqvqgvuzaxdi4/TON_IoT.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/e9nqvqgvuzaxdi4/TON_IoT.csv [following]
--2024-05-16 18:03:18--  https://www.dropbox.com/s/raw/e9nqvqgvuzaxdi4/TON_IoT.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucad7370b1c622da2c589ead7316.dl.dropboxusercontent.com/cd/0/inline/CTBjPYrZqZt5WfhYLvxiW8m_UUa2QIoxOgQsyE68Ch9_5h6_2PKvYvcIWd9cI0ZNfLuBm_C-C4Uub10AlcPOn09i__R6fuhhcw5CS3MLdnjaBKsgs6E86pqVUiesjDq0DuutPPA1g-Dmn_OTrBaDfrgG/file# [following]
--2024-05-16 18:03:19--  https://ucad7370b1c622da2c589ead7316.dl.dropboxusercontent.com/cd/0/inline/CTBjPYrZqZt5WfhYLvxiW8m_UUa2QIoxOgQsyE68Ch9_5h6_2PKvYvcIWd9cI0ZNfLuBm_C-C4Uub10AlcPOn09i__R6fuhhc

# Generate AES key

In [3]:
# Generate an AES key
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

aes_key = AESGCM.generate_key(bit_length=128)  # AES key bit length can be 128, 192, or 256

print("AES key generated successfully.")

AES key generated successfully.


In [4]:
import pandas as pd

df_iot = pd.read_csv('/content/TON_IoT.csv?dl=0')
df_iot

,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,1554198358,3.122.49.24,1883,192.168.1.152,52976,tcp,-,80549.530260,1762852,41933215,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
1,1554198358,192.168.1.79,47260,192.168.1.255,15600,udp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
2,1554198359,192.168.1.152,1880,192.168.1.152,51782,tcp,-,0.000000,0,0,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
3,1554198359,192.168.1.152,34296,192.168.1.152,10502,tcp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
4,1554198362,192.168.1.152,46608,192.168.1.190,53,udp,dns,0.000549,0,298,...,0,0,-,-,-,bad_UDP_checksum,-,F,0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461038,1556340862,192.168.1.32,33108,176.28.50.165,80,tcp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,1,xss
461039,1556423390,192.168.1.33,37242,34.230.157.88,443,tcp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,1,ransomware
461040,1556436603,192.168.1.37,4444,192.168.1.193,49178,tcp,-,290.371539,101568,2592,...,0,0,-,-,-,-,-,-,1,backdoor
461041,1556540442,192.168.1.31,60816,104.98.29.100,443,tcp,-,23.190902,32,31,...,0,0,-,-,-,-,-,-,1,mitm


In [5]:
import os

# AES-GCM setup
aesgcm = AESGCM(aes_key)

# For AES GCM, a nonce is needed. It should be unique but not secret.
# Typically it is stored along with the ciphertext.
nonce = os.urandom(12)

In [6]:
import time

start_time = time.time()

# Read the CSV file as bytes
csv_file_path = '/content/TON_IoT.csv?dl=0'
with open(csv_file_path, 'rb') as file:
    csv_bytes = file.read()

# Encrypt the data
encrypted_data = aesgcm.encrypt(nonce, csv_bytes, None)

encryption_time = time.time() - start_time

print(f"data encrypted successfully in {encryption_time:.6f} seconds.")


data encrypted successfully in 0.360156 seconds.


In [7]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.serialization import load_pem_public_key
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os

# Path to the iot_security folder
folder_path = '/content/drive/My Drive/iot_security/'

# Load the receiver's public key from Google Drive
public_key_path = os.path.join(folder_path, 'public_key.pem')
with open(public_key_path, 'rb') as f:
    public_key_bytes = f.read()
    public_key = load_pem_public_key(public_key_bytes, backend=default_backend())

print("Receiver's public key loaded successfully.")


Receiver's public key loaded successfully.


In [8]:
# Encrypt the AES key
encrypted_aes_key = public_key.encrypt(
    aes_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

print("AES key encrypted successfully.")

AES key encrypted successfully.


In [9]:
encrypted_file_path = '/content/drive/My Drive/iot_security/encrypted_file.enc'
with open(encrypted_file_path, 'wb') as enc_file:
    enc_file.write(encrypted_data)

In [10]:
# Save the encrypted AES key
encrypted_key_path = '/content/drive/My Drive/iot_security/encrypted_aes_key.bin'
with open(encrypted_key_path, 'wb') as file:
    file.write(encrypted_aes_key)

In [11]:
# Save nonce
encrypted_nonce_path = '/content/drive/My Drive/iot_security/nonce.bin'
with open(encrypted_nonce_path, 'wb') as file:
    file.write(nonce)